# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
#Samples are uniformly distributed over the half-open interval [low, high) (includes low, but excludes high)
#size : Output shape: int or tuple of ints, optional
# test size = 150 , actual size 1500, reduces to about 600 unqiue values
lats = np.random.uniform(low=-90.000, high=90.000, size=150)
lngs = np.random.uniform(low=-180.000, high=180.000, size=150)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
#cities
cityCount=len(cities)


In [3]:
#Prepare containers (City	Cloudiness	Country	Date	Humidity	Lat	Lng	Max Temp	Wind Speed)
rcities=[]
rlats=[]
rlngs=[]
clouds=[]
countries=[]
dates=[]
humidities=[]
mxTemps=[]
winds=[]

In [4]:
### Perform API Calls
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
appid = api_key
#query_url = f"{url}appid={api_key}&units={units}&q="+city 
query_url = url+"appid="+api_key+"&units="+units+"&q="

cityCounter=1
for city in cities:
    try:
        city_query_url = query_url+city 
       # print(city_query_url)
        response = requests.get(city_query_url).json()
        rcities.append(response['name'])
        rlats.append(response['coord']['lat'])
        rlngs.append(response['coord']['lon'])
        clouds.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidities.append(response['main']['humidity'])
        mxTemps.append(response['main']['temp_max'])
        winds.append(response['wind']['speed'])
        print("Processing Record "+str( cityCounter)+ " of " +str(cityCount)+" | " + city)
        cityCounter= cityCounter+1
    except:
        print('Skipped '+city)
#print(response)


Processing Record 1 of 118 | mwinilunga
Processing Record 2 of 118 | enderby
Processing Record 3 of 118 | hithadhoo
Processing Record 4 of 118 | allapalli
Processing Record 5 of 118 | georgetown
Processing Record 6 of 118 | san policarpo
Processing Record 7 of 118 | dikson
Processing Record 8 of 118 | ushuaia
Processing Record 9 of 118 | eyl
Processing Record 10 of 118 | vaini
Processing Record 11 of 118 | mataura
Processing Record 12 of 118 | castro
Processing Record 13 of 118 | port alfred
Processing Record 14 of 118 | jamestown
Processing Record 15 of 118 | zhanaozen
Processing Record 16 of 118 | bethel
Processing Record 17 of 118 | narsaq
Skipped asau
Processing Record 18 of 118 | loukhi
Processing Record 19 of 118 | qaanaaq
Processing Record 20 of 118 | saveh
Processing Record 21 of 118 | hofn
Processing Record 22 of 118 | saint-pierre
Skipped barentsburg
Processing Record 23 of 118 | sao filipe
Processing Record 24 of 118 | busselton
Processing Record 25 of 118 | cape town
Proces

In [5]:
#print(len(rcities), len(countries), len(dates),len(rlats),len(rlngs),len(clouds),len(humidities),len(mxTemps),len(winds))

In [8]:
### Convert Raw Data to DataFrame & save
weather_dict = {
    "City": rcities,
    "Country":countries,
    "Date":dates,
    "Lat": rlats,
    "Long": rlngs,
    "Cloudiness": clouds,
    "Humidity": humidities,
    "Max Temp.":mxTemps,
    "Wind Speed": winds
}

weather_data = pd.DataFrame(weather_dict)
now = datetime.now()

filen="weatherData"+now.strftime("%Y%m%d") +".csv"
weather_data.to_csv(filen)

weather_data.head()

,City,Country,Date,Lat,Long,Cloudiness,Humidity,Max Temp.,Wind Speed
0,Mwinilunga,ZM,1544509969,-11.73,24.43,80,93,19.25,2.48
1,Enderby,CA,1544508000,50.55,-119.14,90,80,2.00,2.60
2,Hithadhoo,MV,1544509970,-0.60,73.08,56,100,28.82,4.73
3,Allapalli,IN,1544509970,19.43,80.06,36,78,25.90,3.33
4,Georgetown,GY,1544508000,6.80,-58.16,75,100,22.00,2.78


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot